# 2D Dynamic Programming

Solutions to Leetcode problems that use 2D dynamic programming.

In [9]:
from typing import List

# Problem One: Unique Paths (Medium)

[Leetcode #62](https://leetcode.com/problems/unique-paths/)

- `dp[i][j]` represents the number of unique paths to cell i,j
- `dp[0][0]` is 1 (starting square)
- For each cell i, j:
    - if cell to left exists, add number of unique paths of cell to the left to number of unique paths to this cell
    - if cell above exists, add number of unique paths of cell to the cell above to number of unique paths to this cell
- Return number of unique paths to finish cell

In [1]:
def uniquePaths(m: int, n: int) -> int:
        dp = [[0] * n for i in range(m)]
        
        dp[0][0] = 1

        for i in range(m):
            for j in range(n):
                if i > 0:
                    dp[i][j] += dp[i-1][j]
                if j > 0:
                    dp[i][j] += dp[i][j-1]
        
        return dp[-1][-1]

In [3]:
assert uniquePaths(5, 5) == 70
uniquePaths(50, 25)

11844267374132633700

# Problem Two: Longest Common Subsequence (Medium)

[Leetcode #1143](https://leetcode.com/problems/longest-common-subsequence/)

Fill `dp` with `0`s. Add an extra row and column at the end. Starting from the bottom right (not counting extra row and column), for each cell `(i,j)`:

- If the letters are the same (i.e. `text2[i] == text1[j]`), then `dp[i][j]` is the one plus the value to the bottom right (`dp[i+1][j+1]`)

- Else `dp[i][j]` is the max of the right and bottom cells (`max(dp[i+1][j], dp[i][j+1])`)

Return `dp[0][0]`

In [6]:
def longestCommonSubsequence(text1: str, text2: str) -> int:
        n, m = len(text1), len(text2)

        dp = [[0] * (n+1) for i in range(m+1)]

        for i in range(m - 1, -1, -1):
            for j in range(n - 1, -1, -1):
                if text2[i] == text1[j]:
                    dp[i][j] = dp[i+1][j+1] + 1
                else:
                    dp[i][j] = max(dp[i+1][j], dp[i][j+1])

        return dp[0][0]

In [7]:
assert longestCommonSubsequence('abcde', 'ace') == 3

# Problem Three: Best TIme to Buy and Sell Stock With Cooldown (Medium)

[Leetcode #309](https://leetcode.com/problems/best-time-to-buy-and-sell-stock-with-cooldown/)

Use DP with **caching**. `dp` is our cache and the key is `(i, buying)`. `dp[(i, buying)]` stores the max profit we can make on day `i` with buying state `buying` (this is a boolean). If we are buying, the max profit we can achieve is the max profit of tommorow (`i+1`) where we are NOT buying (`!buying`) minus the cost of buying the stock (`prices[i]`). However we also have the option of not doing anything and moving on to the next day (cooldown, `buying` stays the same). Choose the option with better profit. Else if we are not buying, the max profit we can achieve is the max profit of two days ahead since there is a one day cooldown (`i+2`) when we are buying (`!buying`) plus the cost of selling of the stock today (`prices[i]`). However we also have the option of not doing anything and moving on to the next day (cooldown, `buying` stays the same). Choose the option with better profit.

In [10]:
def maxProfit(prices: List[int]) -> int:
        # Initialize DP cache
        # The key is going to be (i, buying), the value is maxProfit for this state
        dp = {}

        # Define recursive function
        def dfs(i, buying):
            # Base Case 1: Out of bounds
            if i >= len(prices):
                return 0
            
            # Base Case 2: We have already cached the result
            if (i, buying) in dp:
                return dp[(i, buying)]
            
            # Get the max profit of not changing state (i.e. not selling if we bought, not buying if we haven't bought)
            cooldown = dfs(i+1, buying)

            if buying:
                # If we are buying, get the max profit from buying today
                buy = dfs(i + 1, not buying) - prices[i]

                # Set dp[(i, buying)] to max of the options buy and cooldown
                dp[(i, buying)] = max(buy, cooldown)
            else:
                # If we are selling, get the max profit from selling today
                # We skip two days forward since after selling the next day is cooldown
                sell = dfs(i + 2, not buying) + prices[i]

                # Set dp[(i, buying)] to max of the options sell and cooldown
                dp[(i, buying)] = max(sell, cooldown)
            
            return dp[(i, buying)]
        
        return dfs(0, True)

In [11]:
maxProfit([1,2,3,0,2])

3

# Problem Four: Coin Change II (Medium)

[Leetcode #518](https://leetcode.com/problems/coin-change-ii/)

Keep 2D DP array. Rows represent each coin, columns represent `[0, amount]`. The `0` column is filled with ones as there is only 1 way to make `0` coins. Fill each cell `(i, j)` with the sum of the cell `coins[i]` indices to the right (if it is in bounds) and the cell below (if it is in bounds).

Memory optimized solution: Instead of storing each row, we just need one row at a time. Since we were always adding the value of the row below, just build the initial row into a 1D DP array, and keep adding on top of it for each new row. The time complexity is still `O(M*N)` where there are `M` coins and amount is `N`, but the space complexity is only `O(N)` instead of `O(M*N)`.

In [12]:
def change(amount: int, coins: List[int]) -> int:
        dp = [0] * (amount  + 1)
        dp[0] = 1  # 1 way to make amount 0

        for coin in coins:  # Loop through all coins
            for i in range(1, amount + 1):
                if i - coin >= 0:  # If coin doesn't overshoot the current target amount, i
                    dp[i] += dp[i - coin]  # Add the number of ways to make amount i - coin
                    # Only need one DP array, since we keep adding on top of this one row
    
        return dp[-1]